In [11]:
!pip install -q google-genai google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [12]:
import os
import textwrap
import warnings
from datetime import date
from google.colab import userdata
from IPython.display import display, Markdown
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

In [13]:
# Configuração da API
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
client = genai.Client()

In [19]:
warnings.filterwarnings("ignore")
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [20]:
# Função auxiliar para executar agente

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

In [21]:
def criar_agente_explicador(modo_usuario):
    instrucoes = {
        "infantil": "Explique como se o usuário tivesse 5 anos. Use analogias com brinquedos, animais e situações do cotidiano de uma criança.",
        "ensino_medio": "Explique como para um estudante do ensino médio, com exemplos aplicados, mas sem termos muito técnicos.",
        "adulto": "Explique de forma clara, objetiva e técnica, como para um adulto curioso.",
        "fofoca": "Explique como se fosse uma fofoca entre amigos. Use humor, gírias leves e analogias engraçadas como se estivesse contando um babado.",
        "memes": "Explique com tom de meme e linguagem da internet. Use emojis, hashtags e expressões engraçadas."
    }

    return Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction=instrucoes.get(modo_usuario, "Explique de forma clara e divertida."),
        description="Agente que explica conceitos com o estilo desejado."
    )

def agente_quiz(assunto):
    quiz = Agent(
        name="agente_quiz",
        model=MODEL_ID,
        instruction="Crie uma pergunta ou desafio simples para testar se o usuário aprendeu o conteúdo.",
        description="Agente que cria um quiz ou desafio."
    )
    return call_agent(quiz, f"Crie uma pergunta para testar o conhecimento sobre: {assunto}")

def agente_link_alura(assunto):
    link = Agent(
        name="agente_alura",
        model=MODEL_ID,
        instruction="Sugira um curso da Alura que aprofunde esse tema.",
        description="Agente que sugere cursos da Alura."
    )
    return call_agent(link, f"Sugira um curso da Alura para aprender mais sobre: {assunto}")

In [22]:
def aprende_ai(assunto, tempo, modo):
    explicador = criar_agente_explicador(modo)
    prompt = f"Explique o conceito de '{assunto}' em no máximo {tempo} minutos."
    explicacao = call_agent(explicador, prompt)
    quiz = agente_quiz(assunto)
    link = agente_link_alura(assunto)

    print("\n📚 Explicação personalizada:\n")
    display(to_markdown(explicacao))

    print("\n❓ Pergunta rápida para testar aprendizado:\n")
    display(to_markdown(quiz))

    print("\n📎 Para saber mais:\n")
    display(to_markdown(link))

# ---------- Execução principal ---------- #

def iniciar_aprende_ai():
    while True:
        assunto = input("\nQuero aprender sobre: ")
        tempo = input("Tempo que tenho (2, 5, 10, 15 minutos): ")

        print("\n🧠 Como você quer aprender?")
        print("1 - Como se eu tivesse 5 anos 👶")
        print("2 - Como estudante do ensino médio 🧑")
        print("3 - Como adulto curioso 👩‍🎓")
        print("4 - Com analogias e fofoca 🤪")
        print("5 - Em memes 😂")
        print("9 - Surpreenda-me com o estilo! 🎲")

        modo_map = {
            "1": "infantil",
            "2": "ensino_medio",
            "3": "adulto",
            "4": "fofoca",
            "5": "memes"
        }

        modo_escolhido = input("Escolha um número: ")

        if modo_escolhido == "9":
            import random
            modo = random.choice(list(modo_map.values()))
            print(f"✨ Estilo sorteado: {modo}")
        else:
            modo = modo_map.get(modo_escolhido, "adulto")

        aprende_ai(assunto, tempo, modo)

        repetir = input("\n🔁 Quer aprender outro assunto? (s/n): ").lower()
        if repetir != 's':
            print("\nAté a próxima! 👋")
            break

In [23]:
# Executar o app
iniciar_aprende_ai()


Quero aprender sobre: grafos
Tempo que tenho (2, 5, 10, 15 minutos): 15

🧠 Como você quer aprender?
1 - Como se eu tivesse 5 anos 👶
2 - Como estudante do ensino médio 🧑
3 - Como adulto curioso 👩‍🎓
4 - Com analogias e fofoca 🤪
5 - Em memes 😂
9 - Surpreenda-me com o estilo! 🎲
Escolha um número: 9
✨ Estilo sorteado: memes

📚 Explicação personalizada:



> E aí, beleza?! 😎 Se liga que vou te explicar o que são grafos em 15 minutos, no maior estilo "tá fácil, tá tranquilo, tá favorável"! 😜
> 
> **O que são Grafos? 🤔**
> 
> Imagina que você tem um monte de amigos 🧑‍🤝‍🧑 e quer saber quem é amigo de quem. Ou então, um mapa de cidades 🗺️ e quer saber como ir de uma cidade pra outra. É aí que os grafos entram em ação! ✨
> 
> Grafos são tipo um diagrama que mostra conexões entre coisas. Essas "coisas" são chamadas de **vértices** (ou nós), e as conexões entre elas são chamadas de **arestas** (ou links). Pensa nos vértices como bolinhas e nas arestas como linhas ligando as bolinhas. 🤯
> 
> **Tipos de Grafos? 🌈**
> 
> *   **Grafos Não Direcionados:** As arestas não têm uma direção específica. Tipo uma rua de mão dupla. 🔄
> *   **Grafos Direcionados (Dígrafos):** As arestas têm uma direção. Tipo uma rua de mão única. ➡️
> *   **Grafos Ponderados:** As arestas têm um "peso", tipo uma distância ou custo. 💰
> *   **Grafos Não Ponderados:** As arestas não têm peso. 🤷‍♂️
> 
> **Pra que servem os Grafos? 🚀**
> 
> Grafos são tipo canivete suíço! Servem pra um montão de coisas, saca só:
> 
> *   **Redes Sociais:** Pra ver quem é amigo de quem e recomendar amigos. 📱
> *   **Roteamento:** Pra achar o melhor caminho no GPS ou na internet. 📍
> *   **Planejamento de Projetos:** Pra organizar as tarefas e ver a ordem em que devem ser feitas. 🏗️
> *   **Inteligência Artificial:** Pra criar redes neurais e fazer os computadores pensarem. 🧠
> 
> **Exemplo Prático: Indo pra Balada! 💃🕺**
> 
> Digamos que você quer ir pra balada e tem várias opções de caminhos. Cada cruzamento é um vértice e cada rua é uma aresta. Se tiver trânsito, a aresta fica "mais pesada". O grafo te ajuda a achar o caminho mais rápido pra chegar na balada e não perder o fervo! 🔥
> 
> **Resumindo:**
> 
> *   Grafos são conexões entre coisas.
> *   Vértices são as "coisas".
> *   Arestas são as conexões.
> *   Serve pra um montão de coisas!
> 
> E aí, curtiu a explicação? 😎 Agora você já pode se gabar pros seus amigos que manja dos grafos! 😂 #GrafosNaVeia #NoBrainNoGain #PartiuBalada #ConhecimentoÉPoder
> 



❓ Pergunta rápida para testar aprendizado:



> Qual das seguintes alternativas melhor descreve a principal diferença entre um grafo e uma árvore?
> 
> (a) Grafos não podem ter ciclos, enquanto árvores sempre têm.
> (b) Árvores são um tipo específico de grafo que não permite ciclos, enquanto grafos podem conter ciclos.
> (c) Grafos são sempre direcionados, enquanto árvores nunca são.
> (d) Árvores podem ter múltiplas raízes, enquanto grafos têm apenas uma.
> 



📎 Para saber mais:



...


❓ Pergunta rápida para testar aprendizado:



> Qual a principal diferença entre as interfaces `List` e `Set` em Java, e qual caso de uso seria mais adequado para cada uma?
> 



📎 Para saber mais:



> Para se aprofundar em Java, sugiro o curso **"Java e Orientação a Objetos"**. Ele é um excelente ponto de partida para quem quer dominar os fundamentos da linguagem e os princípios da Orientação a Objetos, que são cruciais para construir aplicações robustas e bem estruturadas.
> 
> Nesse curso, você vai aprender desde os conceitos básicos de Java até tópicos mais avançados, como:
> 
> *   Sintaxe e estrutura da linguagem
> *   Classes e objetos
> *   Encapsulamento, herança e polimorfismo
> *   Tratamento de exceções
> *   Collections
> *   E muito mais!
> 
> Com esse curso, você estará preparado para construir seus próprios projetos em Java e dar um grande passo na sua carreira como desenvolvedor.



🔁 Quer aprender outro assunto? (s/n): n

Até a próxima! 👋
